In [546]:
from datetime import datetime, date, timedelta
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import types
from pprint import pprint

class Holiday:
    def __init__(self,name, date):
        self.name = name
        # only accept datetime objects as date
        self.date = datetime.strptime(date, "%Y-%m-%d")
    
    def __str__(self):
        return f"{self.name} ({self.date})"

In [547]:
#Need to uncomment print in addHoliday function

class HolidayList:
    "HolidayList class"
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            # Make sure holidayObj is an Holiday Object by checking the type. Add to innerHolidays
            self.innerHolidays.append(holidayObj)
            #print(f"Added holiday {holidayObj}")

    def findHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            # Find and return Holiday in innerHolidays
            if holiday.name == HolidayName and holiday.date == Date:
                return holiday
            else:
                print(f"{holiday} is not in list.")

    def removeHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            # Find Holiday in innerHolidays by searching the name and date combination.
            if holiday.name == HolidayName and holiday.date == Date:
                # remove the Holiday from innerHolidays, and inform user you deleted the holiday
                self.innerHolidays.remove(holiday)
                print(f"{holiday} was removed from the list.")
            else:
                print(f"{holiday} is not in list.")

    def read_json(self, filelocation):
        # Read in things from json file location and add holidays to innerHolidays
        with open(filelocation, "r") as jsonfile:
            data = json.load(jsonfile)
            for i in range(len(data["holidays"])):
                holiday = Holiday(data["holidays"][i]["name"], data["holidays"][i]["date"])
                HolidayList.addHoliday(self, holiday)

    def save_to_json(self, filelocation):
        json_list = []
        json_dict = {"holidays": json_list}
        # Write out json file to selected file.
        with open(filelocation, "w") as jsonfile:
            for holiday in self.innerHolidays:
                json_list.append(holiday.__dict__)
            json.dump(json_dict, jsonfile, indent=3)

    def scrapeHolidays(self):
        def getHTML(url):
            response = requests.get(url)
            return response.text
        # Scrape holidays from 2020-2024
        for year in range(2020,2025):
            year = str(year)
            html = getHTML("https://www.timeanddate.com/calendar/custom.html?year=" + year +"&country=1&cols=3&df=1&hol=33554809")
            soup = BeautifulSoup(html,'html.parser')
            holiday_data = soup.find('table', attrs = {'class':'cht lpad'}).find('tbody')
            for row in holiday_data:
                items = row.find_all('tr')
                holiday_dict = {}
                holiday_dict['name'] = row.find('td').find_next('td').string
                holiday_dict['date'] = f"{row.find('td').string}, {year}"
                # Check to see if name and date of holiday is in innerHolidays array. If it is, don't add it to list
                if holiday_dict['name'] in self.innerHolidays and holiday_dict['date'] in self.innerHolidays:
                    continue
                else:
                    # converting date to correct format
                    holiday_dict['date'] = str(datetime.strptime(holiday_dict['date'], "%b %d, %Y"))[0:10]
                    # add holiday to list
                    holiday = Holiday(holiday_dict["name"], holiday_dict["date"])
                    HolidayList.addHoliday(self, holiday)
    
    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)

    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and cast results as list
        global filter_by_week
        filter_by_week = filter(lambda holiday:datetime.strftime(holiday.date, "%U") == week_number
            and datetime.strftime(holiday.date, "%Y") == year, self.innerHolidays)
        filter_by_week = list(filter_by_week)
        return(filter_by_week)
    
    def displayHolidaysInWeek(self, year, week_number):
        # Print list of holidays within a week as a parameter
        HolidayList.filter_holidays_by_week(self, year, week_number)
        for row in range(len(filter_by_week)):
            print(filter_by_week[row])
    
    def getWeather_currentweek(self):
        # Query API for weather in current week
        url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"
        querystring = {"q":"new york,us","lat":"41","lon":"74","cnt":"7","units":"imperial"}
        headers = {'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
            'x-rapidapi-key': "1e19830f3bmshf56abb61a6e9168p189818jsnf035a12afc26"}
        weather = requests.request("GET", url, headers=headers, params=querystring)
        weather_dict = weather.json()
        current_week_weather = []
        # return weather string for current week
        for i in range(0,7):
            daily_weather = (weather_dict["list"][i]["weather"][0]["main"])
            current_week_weather.append(daily_weather)
        return(current_week_weather)
    
    def viewCurrentWeek(self):
        # Create list of dates for current week
        today = date.today()
        this_week = []
        for i in range(0,7):
            this_week.append(str(today + timedelta(days=i)))
        print(this_week)
        # Use the Datetime Module to look up current week and year
        current_year = datetime.strftime(today, "%Y")
        current_week = datetime.strftime(today, "%U")
        # Use your filter_holidays_by_week function to get the list of holidays for the current week and year
        HolidayList.filter_holidays_by_week(self, current_year, current_week)
        # Use your displayHolidaysInWeek function to display the holidays in the week
        HolidayList.displayHolidaysInWeek(self, current_year, current_week)
        # Ask user if they want to get the weather
        weather_yn  = str(input("Would you like to see this week's weather? [y/n]"))
        while True:
            weather_yn  = str(input("Would you like to see this week's weather? [y/n]"))
            # If yes, use your getWeather function and display results
            if weather_yn == "y":
                current_week_weather = HolidayList.getWeather_currentweek(self)
                current_weather_date = {}
                for i in range(0,7):
                    current_weather_date['date'] = this_week[i]
                    current_weather_date['weather'] = current_week_weather[i]
                    print(current_weather_date)
                break
            elif weather_yn == "n":
                break
            else:
                print("I didn't understand. Try again.")

In [ ]:
# 5. Display User Menu (Print the menu)
    # 6. Take user input for their action based on Menu and check the user input for errors
    # 7. Run appropriate method from the HolidayList object depending on what the user input is
    # 8. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 
        # 4-8: Use code from Tournament Track

In [555]:
def main():
    # 1. Initialize HolidayList Object
    holidaylist = HolidayList()
    # 2. Load JSON file via HolidayList read_json function
    HolidayList.read_json("holidays.json")
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    holidaylist.scrapeHolidays()
    # 4. Create while loop for user to keep adding or working with the Calender

    #Start Up
    print(f"""Holiday Management
    ============================
    There are {holidaylist.numHolidays()} holidays stored in the system.""")

# #create dictionary
# dict={}
# for num in range(1,total_slots+1):
#     dict[num] = None

# #Main Menu
# print("""
# Participant Menu
# ================
# 1. Sign Up
# 2. Cancel Sign Up
# 3. View Participants
# 4. Save Changes
# 5. Exit
# """)

# exit = "n"

# name = str
# slot = int

# def name_slot():
#     global name
#     global slot
#     name = str(input("Participant Name"))
#     slot = int(input(f"Starting Slot #[1-{total_slots}]: "))

# while exit != "y":
#     menu_selection = int(input("Enter 1 for Sign Up.\nEnter 2 for Cancel Sign Up.\nEnter 3 for View Participants.\nEnter 4 for Save Changes.\nEnter 5 for Exit.\n"))
#     if menu_selection == 1:
#         print("Participant Sign Up\n====================")
#         name_slot()
#         print(f"Participant Name: {name}")
#         print(f"Desired starting slot #[1-{total_slots}]: {slot}\n")
#         if dict[slot] == None:
#             dict[slot] = name
#             print(f"Success:\n{name} is signed up in starting slot #{slot}.\n")
#             menu_selection == 0
#         else:
#             print(f"Error:\nSlot #{slot} is filled. Please try again.\n")
#     elif menu_selection == 2:
#         print("Participant Cancellation\n====================")
#         name_slot()
#         print(f"Participant Name: {name}")
#         print(f"Starting slot #[1-{total_slots}]: {slot}\n")
#         if dict[slot] == name:
#             dict[slot] = None
#             print(f"Success:\n{name} has been cancelled from starting slot #{slot}.\n")
#             menu_selection == 0
#         else:
#             print(f"Error:\n{name} is not in that starting slot. Please try again.\n")
#     elif menu_selection == 3:
#         print("View Participants\n=================")
#         slot = int(input("Slot Number"))
#         print(f"Starting slot #[1-{total_slots}]: {slot}\nStarting Slot: Participant")
#         if slot <= 5:
#             for slot in range(1, slot+6):
#                 print(f"{slot}: {dict[slot]}")
#         elif slot >= (total_slots-5):
#             for slot in range(total_slots-5, total_slots+1):
#                 print(f"{slot}: {dict[slot]}")
#         elif slot in range(6, total_slots-6):
#             for slot in range(slot-5, slot+6):
#                 print(f"{slot}: {dict[slot]}")
#         else:
#             print(f"Error:\n{slot} is not a starting slot.")
#     elif menu_selection == 4:
#         print("Save Changes\n============")
#         save= str(input("Save your changes to CSV? [y/n]"))
#         print(f"Save your changes to CSV? [y/n]: {save}\n")
#         if save == "y":
#             slot=1
#             f=open("tournament-tracker.csv", "w")
#             for slot in range(1,total_slots+1):
#                 f.write(f"{slot}: {dict[slot]}\n")
#             f.close()
#             print("Changes saved.")
#         elif save == "n":
#             print("Changes not saved.")
#         else:
#             print(f"Error: {save} is not an option\nSave your changes to CSV? [y/n]")
#     elif menu_selection == 5:
#         print("Exit\n=====\nAny unsaved changes will be lost.")
#         exit = str(input("Are you sure you want to exit? [y/n]"))
#         print(f"Are you sure you want to exit? [y/n]: {exit}\n")
#         if exit == "y":
#             print("Goodbye!")
#             break
#         elif exit == "n":
#             print("Okay. Not exiting.")
#         else:
#             print(f"Error: {exit} is not an option\nAre you sure you want to exit? [y/n]")
#     else:
#         print("Please select a menu option from 1 through 5.")


Holiday Management
There are 7 stored in the system.


In [554]:
holiday1 = Holiday("New Year's Day", "2022-01-01")
print(holiday1)

holidaylist = HolidayList()

#holidaylist.addHoliday(holiday1)
#holidaylist.findHoliday("New Year's Day", "2022-01-01")
#holidaylist.removeHoliday("New Year's Day", "2022-01-01")

holidaylist.read_json("holidays.json")

#holidaylist.scrapeHolidays()

holidaylist.numHolidays()

#holidaylist.displayHolidaysInWeek("2022", "00")
#holidaylist.getWeather_currentweek()

#holidaylist.viewCurrentWeek()




New Year's Day (2022-01-01 00:00:00)


7